# Import Libraries

In [1]:
import yfinance as yf
import pandas as pd
import yesg
from datetime import datetime
import numpy as np
from tqdm import trange


# Récupération des données

On cherche les tickers des entreprises Néerlandaises côtées en bourse. Ainsi, grâce au fichier csv nous obtenons :

In [2]:
tickers = pd.read_csv("./datas/Euronext_Equities_2022-12-02.csv", sep=";")
tickers_amsterdam = tickers[tickers['Currency']=='EUR']['Symbol'].tolist()
for i in range(len(tickers_amsterdam)):
    tickers_amsterdam[i] = tickers_amsterdam[i] + ".AS"
print(f"Nous avons : {len(tickers_amsterdam)}, actions")

Nous avons : 168, actions


## Récupération des scores ESG

Pour la construction de notre portefeuille d'actions nous avons besoin des scores ESG de toutes les entreprises disponibles.

In [16]:
esg_scores = pd.DataFrame(columns = ['Ticker Yahoo', 'Environment Score', 'Social Score', 'Governance Score', 'Total Score'], index = range(len(datas['Close'].columns)))

for i in trange(len(datas['Close'].columns)):
    ticker = tickers_amsterdam[i]
    try:
        sus = yf.Ticker(ticker).sustainability
        scores = sus.loc[['environmentScore','socialScore','governanceScore','totalEsg'],'Value']
        esg_scores.loc[i] = [ticker, scores[0], scores[1], scores[2], scores[3]]
    except:
        pass
esg_scores

100%|██████████| 168/168 [15:44<00:00,  5.62s/it]


,Ticker Yahoo,Environment Score,Social Score,Governance Score,Total Score
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
163,NaN,NaN,NaN,NaN,NaN
164,VPK.AS,10.56,8.74,3.75,23.05
165,NaN,NaN,NaN,NaN,NaN
166,NaN,NaN,NaN,NaN,NaN


In [18]:
for i in range(len(datas['Close'].columns)):
    ticker = tickers_amsterdam[i]
    esg_scores.loc[i, 'Ticker Yahoo'] = ticker

100%|██████████| 168/168 [00:00<00:00, 20420.29it/s]


Sauvegardons ce fichier dans le dossier datas.

In [20]:
esg_scores.to_csv('./datas/esg_scores.csv')
esg_scores.head()

Nous voyons bien qu'il manque énormément de score ESG, nous allons donc être obligés d'aller chercher à la main les scores restant.

## Récupération des prix

Désormais récupérons le prix de toutes les actions disponibles.

In [3]:
tickers = yf.Tickers(tickers_amsterdam)
datas = tickers.history(period='max')
datas.index = pd.to_datetime(datas.index)

[*********************100%***********************]  168 of 168 completed

26 Failed downloads:
- DSC2S.AS: No data found, symbol may be delisted
- EHCW.AS: No data found, symbol may be delisted
- HEGAW.AS: No data found, symbol may be delisted
- SPR1W.AS: No data found, symbol may be delisted
- BHNDW.AS: No data found, symbol may be delisted
- FAGR.AS: No data found, symbol may be delisted
- NAIW.AS: No data found, symbol may be delisted
- VAMW.AS: No data found, symbol may be delisted
- ENTPT.AS: No data found, symbol may be delisted
- BHNDT.AS: No data found, symbol may be delisted
- SGO.AS: No data found, symbol may be delisted
- EPICW.AS: No data found, symbol may be delisted
- ADUX.AS: No data found, symbol may be delisted
- ENX.AS: No data found for this date range, symbol may be delisted
- EHCT.AS: No data found, symbol may be delisted
- SPR1T.AS: No data found, symbol may be delisted
- NAITR.AS: No data found, symbol may be delisted
- ESGT.AS: No data found, symbol may be delis

Prenons uniquement le prix de fermeture ('Close').

In [5]:
datas_price = datas['Close']

In [63]:
data_from = pd.Timestamp('2016-01-01')
data_filter = datas_price.loc[data_from:]
data_filter.head()

,AALB.AS,ABN.AS,ACOMO.AS,AD.AS,ADUX.AS,ADYEN.AS,AED.AS,AF.AS,AGN.AS,AJAX.AS,...,VAMW.AS,VASTN.AS,VEON.AS,VLK.AS,VPK.AS,VTA.AS,VVY.AS,WDP.AS,WHA.AS,WKL.AS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,25.631207,12.842031,17.596754,15.247333,NaN,NaN,NaN,1.0,3.309674,8.202633,...,NaN,23.892433,NaN,11.392182,32.685501,3.172482,25.982430,NaN,27.354794,26.405645
2016-01-05,25.605684,12.817178,17.596754,15.321725,NaN,NaN,NaN,1.0,3.286875,8.135081,...,NaN,24.119980,NaN,11.512735,32.726620,3.142600,25.982430,NaN,27.549374,26.646494
2016-01-06,25.018612,12.506533,17.516768,15.223839,NaN,NaN,NaN,1.0,3.274499,8.135081,...,NaN,23.920872,NaN,11.400043,31.628881,3.167501,25.977528,NaN,27.652065,26.204212
2016-01-07,24.678276,12.236271,16.949253,15.145526,NaN,NaN,NaN,1.0,3.150085,8.106131,...,NaN,23.383301,NaN,11.216595,30.547588,3.132638,25.541218,NaN,27.025085,26.878588
2016-01-08,24.614470,12.407126,16.914974,15.176853,NaN,NaN,NaN,1.0,3.098626,8.212283,...,NaN,23.252455,NaN,11.137973,30.124109,3.087816,24.805866,NaN,26.827803,26.335583


Supprimons les colonnes avec un NaN à la fin, car elles ne sont plus échangées sur les marchés financiers. Ou bien lorsqu'elles sont échangées depuis trop peu de temps. Nous n'avons pas assez de recul sur ces actions.

In [68]:
last_date = data_filter.index.to_list()[-1]
first_date = data_filter.index.to_list()[0]
last_row_NaN = pd.Series.to_frame(data_filter.iloc[-1].isna())
first_row_NaN = pd.Series.to_frame(data_filter.iloc[0].isna())
missing_price_end = last_row_NaN.index[last_row_NaN[last_date]==True].to_list()
missing_price_begin = first_row_NaN.index[first_row_NaN[first_date]==True].to_list()
try:
    missing_price_begin.remove(missing_price_end)
except:
    pass
data_filter = data_filter.drop(missing_price_end, axis=1)
data_filter = data_filter.drop(missing_price_begin, axis=1)

Sauvegardons ce fichier et voici un apperçu du DataFrame que nous obtenons :

In [72]:
data_filter.to_csv('./datas/prices.csv')
data_filter.head()


,AALB.AS,ABN.AS,ACOMO.AS,AD.AS,AGN.AS,AJAX.AS,AKZA.AS,ALX.AS,AMG.AS,AMUND.AS,...,TWEKA.AS,URW.AS,VALUE.AS,VASTN.AS,VLK.AS,VPK.AS,VTA.AS,VVY.AS,WHA.AS,WKL.AS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,25.631207,12.842031,17.596754,15.247333,3.309674,8.202633,44.708950,2.75,7.972379,2.441340,...,29.775803,150.320496,6.087242,23.892433,11.392182,32.685501,3.172482,25.982430,27.354794,26.405645
2016-01-05,25.605684,12.817178,17.596754,15.321725,3.286875,8.135081,44.560013,2.94,7.724083,2.431610,...,29.816362,152.120178,6.061216,24.119980,11.512735,32.726620,3.142600,25.982430,27.549374,26.646494
2016-01-06,25.018612,12.506533,17.516768,15.223839,3.274499,8.135081,43.919621,2.80,7.699881,2.344040,...,29.224096,151.236694,6.054234,23.920872,11.400043,31.628881,3.167501,25.977528,27.652065,26.204212
2016-01-07,24.678276,12.236271,16.949253,15.145526,3.150085,8.106131,44.046207,2.49,7.378977,2.299813,...,28.473614,147.964600,6.016778,23.383301,11.216595,30.547588,3.132638,25.541218,27.025085,26.878588
2016-01-08,24.614470,12.407126,16.914974,15.176853,3.098626,8.212283,43.100506,2.99,7.350293,2.211359,...,28.238331,146.001343,5.873310,23.252455,11.137973,30.124109,3.087816,24.805866,26.827803,26.335583


# Pre-processing

We have to follow few steps :

* Analyse the liquidity of all firms
    * Market capitalization
    * Average daily volume exchange
    * Free float part
* ESG filter
    * exclude x% of firms with the worts ESG score
    * keep firms with the best ESG momentum
    * take a specific KPI
* Financial analysis
    * Profit Margin
    * Return on assets

We can also analyse the correlation between our chosen stocks.

Then after that we have stocks we will use in our portfolio we need to find best weights. We will use two different methods :
* Mean variance method 
* Black litterman method